<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Instructions" data-toc-modified-id="Instructions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Instructions</a></span></li><li><span><a href="#Workflow-Summary" data-toc-modified-id="Workflow-Summary-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Workflow Summary</a></span></li><li><span><a href="#Preliminaries:-library-imports" data-toc-modified-id="Preliminaries:-library-imports-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Preliminaries: library imports</a></span></li><li><span><a href="#Data-loading" data-toc-modified-id="Data-loading-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data loading</a></span></li><li><span><a href="#Data-dictionary" data-toc-modified-id="Data-dictionary-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Data dictionary</a></span><ul class="toc-item"><li><span><a href="#location" data-toc-modified-id="location-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span><code>location</code></a></span></li><li><span><a href="#psgc" data-toc-modified-id="psgc-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span><code>psgc</code></a></span></li><li><span><a href="#results-tables:-president-and-vice_president" data-toc-modified-id="results-tables:-president-and-vice_president-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>results tables: <code>president</code> and <code>vice_president</code></a></span><ul class="toc-item"><li><span><a href="#unlabeled-columns" data-toc-modified-id="unlabeled-columns-5.3.1"><span class="toc-item-num">5.3.1&nbsp;&nbsp;</span>unlabeled columns</a></span><ul class="toc-item"><li><span><a href="#col5" data-toc-modified-id="col5-5.3.1.1"><span class="toc-item-num">5.3.1.1&nbsp;&nbsp;</span><code>col5</code></a></span></li><li><span><a href="#col7,-col8" data-toc-modified-id="col7,-col8-5.3.1.2"><span class="toc-item-num">5.3.1.2&nbsp;&nbsp;</span><code>col7</code>, <code>col8</code></a></span></li></ul></li></ul></li><li><span><a href="#granularity-of-information" data-toc-modified-id="granularity-of-information-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>granularity of information</a></span><ul class="toc-item"><li><span><a href="#municipal-level-information" data-toc-modified-id="municipal-level-information-5.4.1"><span class="toc-item-num">5.4.1&nbsp;&nbsp;</span>municipal level information</a></span></li><li><span><a href="#precinct-level-information" data-toc-modified-id="precinct-level-information-5.4.2"><span class="toc-item-num">5.4.2&nbsp;&nbsp;</span>precinct level information</a></span></li><li><span><a href="#candidate-information---precinct-votes-breakdown" data-toc-modified-id="candidate-information---precinct-votes-breakdown-5.4.3"><span class="toc-item-num">5.4.3&nbsp;&nbsp;</span>candidate information - precinct votes breakdown</a></span></li></ul></li></ul></li><li><span><a href="#Cleaning:-location-files" data-toc-modified-id="Cleaning:-location-files-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Cleaning: location files</a></span><ul class="toc-item"><li><span><a href="#first-pass-cleaning-:-exact-matching" data-toc-modified-id="first-pass-cleaning-:-exact-matching-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>first pass cleaning : exact matching</a></span></li><li><span><a href="#second-pass-cleaning:-manual-replacement" data-toc-modified-id="second-pass-cleaning:-manual-replacement-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>second pass cleaning: manual replacement</a></span></li><li><span><a href="#third-pass-cleaning:-fuzzy-matching" data-toc-modified-id="third-pass-cleaning:-fuzzy-matching-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>third pass cleaning: fuzzy matching</a></span></li><li><span><a href="#export-location-to-gadm-lookup" data-toc-modified-id="export-location-to-gadm-lookup-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>export location to gadm lookup</a></span></li></ul></li><li><span><a href="#Create-denormalized-tables" data-toc-modified-id="Create-denormalized-tables-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Create denormalized tables</a></span></li></ul></div>

# Instructions

Datasets:
* `ref_table_precinct_locations_PSGC.csv` – lookup table for precincts
* `results_president.csv` – precinct-level election results for the 2016 presidential race
* `results_vice-president.csv` – precinct-level election results for the 2016 vice presidential race

Tasks:
1. Create a denormalized table replacing precinct_code in the results_*.csv files with the columns: region, province, municipality, and barangay. 

2. Create an interesting data visualization using this dataset.

# Workflow Summary

1. Initial data inspection and data dictionary building
2. Data cleaning
3. EDA

Since these datasets are rich with geodata, we'll use GADM shape files for plotting later on. This entails cleaning the location tags to match with the GADM labels. The general workflow for cleaning is divided in to three parts:
1. Exact matching by merging the shape files and the location dataframe
2. Manual cleaning for some items, especially those tagged as overseas votes since they are easily identifiable and we're sure that they're not in our shape files.
3. Fuzzy matching for the rest of the entries.

The result after these steps is a lookup table of precinct codes with the correct GADM location tags so we can easily merge them in Tableau for visualization, and a denormalized table for the `results` file so we can do preliminary EDA on the dataset.

# Preliminaries: library imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#geodata
import geopandas as gpd

#utilities
import os
import shutil
from glob import glob
from operator import itemgetter
from collections import defaultdict
import json
from tqdm import tqdm

#for fuzzy maching
from fuzzywuzzy import process

# spark imports
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import udf
import pyspark.sql.functions as F
from pyspark.sql import Row

sc = SparkContext()
spark = SparkSession(sc)

In [2]:
try:
    os.mkdir(os.path.join('..', 'output'))
except:
    print('folder already exists!')

In [14]:
def rename_files():
    files = glob(os.path.join('..', 'output','*','*csv'))
    filenames = list(map(lambda x: x.split(os.sep)[2]+r'.csv', files))
    for path, filename in tqdm(zip(files, filenames)):
        try:
            ogfilename = os.path.split(path)[-1]
            shutil.move(path, os.path.join('..', 'output'))
            os.rename(os.path.join('..', 'output', ogfilename), os.path.join('..', 'output', filename))
            shutil.rmtree(os.path.join('..', 'output', filename.split('.')[0]))
        except FileNotFoundError:
            pass

In [15]:
def save_query_to_csv(query, filename):
    """Accepts a query and saves it into a single csv file"""
    path = os.path.join('..', 'output', filename)
    try:
        spark.sql(query
                 ).coalesce(1).write.option(
            "header", "true").csv(path)
        rename_files()
        print('file saved!')
    except:
        inp = input('file already exists! retry save? Y/N')
        if inp =='Y':
            shutil.rmtree(path)
            save_query_to_csv(query, filename)
        else:
            pass

# Data loading

In [5]:
shape_files = sorted(glob(os.path.join('..', 'gadm', "gadm*.shp")))[1:]
data_files = sorted(glob(os.path.join('..', 'datasets', '*.csv')))

In [6]:
datasets = dict(map(lambda x: (x.strip(r'.csv').split('_')[-1].lower().replace('-', '_'), x), data_files))
datasets

{'location': '../datasets/ref_table_precinct_locations.csv',
 'psgc': '../datasets/ref_table_precinct_locations_PSGC.csv',
 'president': '../datasets/results_president.csv',
 'vice_president': '../datasets/results_vice-president.csv'}

In [7]:
shapes = dict(zip(['provincial', 'municipal', 'brgy'], shape_files))
shapes

{'provincial': '../gadm/gadm36_PHL_1.shp',
 'municipal': '../gadm/gadm36_PHL_2.shp',
 'brgy': '../gadm/gadm36_PHL_3.shp'}

In [8]:
for label, filepath in tqdm(datasets.items()):
    datasets[label] = spark.read.csv(filepath, sep=',', header=True)
#     print(label)
#     display(datasets[label].limit(2).toPandas())
    datasets[label].createOrReplaceTempView(label)

100%|██████████| 4/4 [00:06<00:00,  1.73s/it]


In [9]:
for label, filepath in tqdm(shapes.items()):
#     print(label)
    gdf = gpd.GeoDataFrame.from_file(filepath)
    col_include = list(filter(lambda x: x.startswith('NAME') or x.startswith('VAR'), gdf.columns))
    df = pd.DataFrame(gdf[col_include])
    for col in df.columns:
        df[col] = df[col].astype(str) 
    shapes[label] = spark.createDataFrame(df)
    shapes[label].createOrReplaceTempView(label)
#     display(shapes[label].limit(2).toPandas())

100%|██████████| 3/3 [00:06<00:00,  2.20s/it]


In [10]:
spark.sql("SHOW TABLES").show()

+--------+--------------+-----------+
|database|     tableName|isTemporary|
+--------+--------------+-----------+
|        |          brgy|       true|
|        |      location|       true|
|        |     municipal|       true|
|        |     president|       true|
|        |    provincial|       true|
|        |          psgc|       true|
|        |vice_president|       true|
+--------+--------------+-----------+



# Data dictionary

First, we check the available data fields and the granularity of the information available. By building a data dictionary, we also establish the relationship of the fields from different datasets so we can decide which information is relevant in building our denormalized OLAP table. If there are unlabled columns, we also try to resolve those in this section. Lastly, we map out the hierarchy of data available to us in order to easily visualize the overview of our datasets.

In [11]:
datasets.keys()

dict_keys(['location', 'psgc', 'president', 'vice_president'])

## `location`
* filename : `ref_table_precinct_locations.csv`
* `registered_voters` and `ballots_cast` are both per precinct
* multiple precinct codes per barangay

In [16]:
datasets['location'].printSchema()

root
 |-- precinct_code: string (nullable = true)
 |-- region: string (nullable = true)
 |-- province: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- barangay: string (nullable = true)
 |-- registered_voters: string (nullable = true)
 |-- ballots_cast: string (nullable = true)



In [17]:
datasets['location'].show(2)

+-------------+--------+----------+------------+--------+-----------------+------------+
|precinct_code|  region|  province|municipality|barangay|registered_voters|ballots_cast|
+-------------+--------+----------+------------+--------+-----------------+------------+
|     55170026|REGION I|PANGASINAN|    CALASIAO| BUENLAG|              592|         503|
|     55170027|REGION I|PANGASINAN|    CALASIAO| BUENLAG|              526|         458|
+-------------+--------+----------+------------+--------+-----------------+------------+
only showing top 2 rows



## `psgc`

* filename: `ref_table_precinct_locations_PSGC.csv`

**NOTES**
* PSGC = [Philippine Standard Geographic Code](https://psa.gov.ph/classification/psgc/)
* same fields as `location`:
```python
 |-- precinct_code: string (nullable = true)
 |-- region: string (nullable = true)
 |-- province: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- barangay: string (nullable = true)
 |-- registered_voters: string (nullable = true)
 |-- ballots_cast: string (nullable = true)
```

* additional fields:
    * `PSGC_CM`, `province_CM`, `municipality_CM` - PSGC-specific code, goes down to barangay level
    * `is_city` - flag for city, 1 if city, 0 if not
    * `income_class` - municipal level
    * `population` - population, municipal level
    * `land_area` - municipal level; unit: $m^2$


In [18]:
datasets['psgc'].printSchema()

root
 |-- _c0: string (nullable = true)
 |-- precinct_code: string (nullable = true)
 |-- region: string (nullable = true)
 |-- province: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- barangay: string (nullable = true)
 |-- registered_voters: string (nullable = true)
 |-- ballots_cast: string (nullable = true)
 |-- PSGC_CM: string (nullable = true)
 |-- is_city: string (nullable = true)
 |-- income_class: string (nullable = true)
 |-- population: string (nullable = true)
 |-- land_area: string (nullable = true)
 |-- province_CM: string (nullable = true)
 |-- municipality_CM: string (nullable = true)



In [21]:
datasets['psgc'].limit(2).toPandas()

,_c0,precinct_code,region,province,municipality,barangay,registered_voters,ballots_cast,PSGC_CM,is_city,income_class,population,land_area,province_CM,municipality_CM
0,0,55170026,REGION I,PANGASINAN,CALASIAO,BUENLAG,592,503,15517000,0.0,1st,91109.0,4836.0,PANGASINAN,CALASIAO
1,1,55170027,REGION I,PANGASINAN,CALASIAO,BUENLAG,526,458,15517000,0.0,1st,91109.0,4836.0,PANGASINAN,CALASIAO


## results tables: `president` and `vice_president`
filename: `results_*.csv`
* `precinct_code`
* `contest_code` - position-specific code, same for all candidates running in the same position
* `candidate_name` - candidate name; code inside the parentheses indicate party affiliation
* `party_code` - numerical code for party affiliation
* `votes` - number of votes garnered in precinct
* `col5` - candidate code; numerical identifier, distinct per candidate
* `ballots_cast` - number of total ballots casted in precinct
* `col7` - 
* `col8` - 
* `timestamp` - time of upload of electoral results per precinct (should be one value per precinct)
* `pct_votes` - `votes`/`ballots_cast`

In [22]:
datasets['president'].printSchema()

root
 |-- precinct_code: string (nullable = true)
 |-- contest_code: string (nullable = true)
 |-- candidate_name: string (nullable = true)
 |-- party_code: string (nullable = true)
 |-- votes: string (nullable = true)
 |-- col5: string (nullable = true)
 |-- ballots_cast: string (nullable = true)
 |-- col7: string (nullable = true)
 |-- col8: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- pct_votes: string (nullable = true)



### unlabeled columns

#### `col5`
* candidate code - unique per candidate

In [24]:
spark.sql("""SELECT DISTINCT col5, candidate_name FROM president""").show()

+----+--------------------+
|col5|      candidate_name|
+----+--------------------+
|   4|    POE, GRACE (IND)|
|   1|   BINAY, JOJO (UNA)|
|   5|ROXAS, MAR DAANG ...|
|   6|SEÑERES, ROY (WPP...|
|   2|DEFENSOR SANTIAGO...|
|   3|DUTERTE, RODY (PD...|
+----+--------------------+



#### `col7`, `col8`
* unique per precinct.

In [25]:
spark.sql('''
            SELECT 
                precinct_code,
                count(col7),
                count(col8)
            FROM (SELECT DISTINCT precinct_code, col7, col8 FROM president)
            GROUP BY precinct_code
            ORDER BY count(col7) DESC, count(col8) DESC
            LIMIT 5
            ''').show()

+-------------+-----------+-----------+
|precinct_code|count(col7)|count(col8)|
+-------------+-----------+-----------+
|     24020457|          1|          1|
|     22170586|          1|          1|
|     72110015|          1|          1|
|      4050001|          1|          1|
|     34050056|          1|          1|
+-------------+-----------+-----------+



* could be number of abstain and invalid votes

In [26]:
spark.sql('''
        SELECT 
            precinct_code,
            SUM(votes) total_votes,
            ballots_cast,
            col7,
            col8,
            SUM(votes) + col7 + col8 sum_all,
            ballots_cast - (SUM(votes) + col7 + col8) diff
        FROM
            president
        GROUP BY
            precinct_code, 
            ballots_cast, 
            col7, col8 
        ORDER BY diff
        LIMIT 20
            ''').show()

+-------------+-----------+------------+----+----+-------+----+
|precinct_code|total_votes|ballots_cast|col7|col8|sum_all|diff|
+-------------+-----------+------------+----+----+-------+----+
|     30210009|      279.0|         300|   7|  14|  300.0| 0.0|
|     70020039|      286.0|         289|   2|   1|  289.0| 0.0|
|     38120006|      537.0|         574|  15|  22|  574.0| 0.0|
|     10320013|      507.0|         526|  12|   7|  526.0| 0.0|
|     71080017|      394.0|         425|  28|   3|  425.0| 0.0|
|     39101260|      509.0|         515|   3|   3|  515.0| 0.0|
|     58130129|      529.0|         534|   4|   1|  534.0| 0.0|
|     21190064|      410.0|         417|   6|   1|  417.0| 0.0|
|      8060009|      375.0|         383|   5|   3|  383.0| 0.0|
|     36030018|      133.0|         137|   2|   2|  137.0| 0.0|
|     56190045|      359.0|         364|   5|   0|  364.0| 0.0|
|     14230094|      520.0|         533|   6|   7|  533.0| 0.0|
|     14190015|      548.0|         571|

## granularity of information

```python
|-- region
|---- province
|------ municipality
|      |--income_class
|      |--population
|      |--land_area
|      |--is_city
|-------- barangay
|---------- precinct
|           |--precinct_code
|           |--registered_voters
|           |--ballots_cast
|           |--timestamp
|           |--invalid_votes
|           |--abstain
|...............candidates (party_code)
               |-- votes
               |-- pct_votes

```


from `location` table:
* `precinct_code`
* `region`
* `province`
* `municipality`
* `barangay`
* `registered_voters`
* `ballots_cast`

from `psgc` table:
* `is_city`
* `income_class`
* `population`
* `land_area`

from results tables (`president` and `vice_president`)
* `candidate_name`
* `party_code`
* `votes`
* `col7`
* `col8`
* `timestamp`
* `pct_votes`

### municipal level information

In [28]:
spark.sql('''
            SELECT DISTINCT
                region, 
                province,
                municipality, 
                is_city, 
                income_class, 
                population, 
                land_area 
            FROM psgc
            LIMIT 5''').show()

+-----------+------------------+-------------------+-------+-----------------+----------+---------+
|     region|          province|       municipality|is_city|     income_class|population|land_area|
+-----------+------------------+-------------------+-------+-----------------+----------+---------+
|REGION IV-A|            LAGUNA|            CAVINTI|    0.0|              3rd|   20809.0|  20358.0|
|  REGION VI|            ILOILO|        ILOILO CITY|    1.0|1st Class-Special|      null|   7834.0|
|       ARMM|       MAGUINDANAO|DATU SAUDI AMPATUAN|    0.0|              4th|   20330.0|   6016.0|
|   REGION X|          BUKIDNON|            DAMULOG|    0.0|              4th|   25538.0|  24419.0|
|   REGION X|MISAMIS OCCIDENTAL|             CLARIN|    0.0|              4th|   35573.0|   8450.0|
+-----------+------------------+-------------------+-------+-----------------+----------+---------+



### precinct level information

**precinct geomapping**

In [30]:
spark.sql('SELECT precinct_code, region, province, municipality, barangay FROM location LIMIT 2').show()

+-------------+--------+----------+------------+--------+
|precinct_code|  region|  province|municipality|barangay|
+-------------+--------+----------+------------+--------+
|     55170026|REGION I|PANGASINAN|    CALASIAO| BUENLAG|
|     55170027|REGION I|PANGASINAN|    CALASIAO| BUENLAG|
+-------------+--------+----------+------------+--------+



**voting statistics per precinct**

In [31]:
spark.sql("""
    SELECT DISTINCT
        l.precinct_code, 
        l.registered_voters, 
        l.ballots_cast,
        p.col7 abstain_votes,
        p.col8 invalid_votes,
        p.timestamp
    FROM location l
    JOIN president p
    ON p.precinct_code = l.precinct_code
    LIMIT 10
""").show()

+-------------+-----------------+------------+-------------+-------------+-------------------+
|precinct_code|registered_voters|ballots_cast|abstain_votes|invalid_votes|          timestamp|
+-------------+-----------------+------------+-------------+-------------+-------------------+
|     58080245|              744|         507|            5|            5|05/09/2016 17:10:49|
|      6110003|              312|         259|            5|            2|05/09/2016 17:11:10|
|     31150045|              274|         231|            5|            6|05/09/2016 17:12:13|
|     19030021|              483|         384|            4|           23|05/09/2016 17:20:45|
|     56450073|              630|         554|            7|            3|05/09/2016 17:32:40|
|     15290029|              747|         634|           10|            5|05/09/2016 17:32:35|
|     37010029|              511|         416|           30|           21|05/09/2016 17:32:34|
|     24021077|              644|         529|    

### candidate information - precinct votes breakdown

In [32]:
spark.sql("""
    SELECT DISTINCT
        p.precinct_code, 
        p.candidate_name,
        p.party_code,
        p.votes,
        p.pct_votes
    FROM president p
    ORDER BY precinct_code
    LIMIT 10
""").show()

+-------------+--------------------+----------+-----+---------------+
|precinct_code|      candidate_name|party_code|votes|      pct_votes|
+-------------+--------------------+----------+-----+---------------+
|     10010001|SEÑERES, ROY (WPP...|       165|    0|            0.0|
|     10010001|   BINAY, JOJO (UNA)|       163|  117| 0.267734553776|
|     10010001|    POE, GRACE (IND)|        58|   59| 0.135011441648|
|     10010001|ROXAS, MAR DAANG ...|        85|   92| 0.210526315789|
|     10010001|DUTERTE, RODY (PD...|       114|  136| 0.311212814645|
|     10010001|DEFENSOR SANTIAGO...|       135|   24|0.0549199084668|
|     10010002|   BINAY, JOJO (UNA)|       163|  115| 0.213358070501|
|     10010002|DUTERTE, RODY (PD...|       114|  156| 0.289424860853|
|     10010002|    POE, GRACE (IND)|        58|  104| 0.192949907236|
|     10010002|SEÑERES, ROY (WPP...|       165|    0|            0.0|
+-------------+--------------------+----------+-----+---------------+



# Cleaning: location files 

Since most of our location specific information outside of the votes per precinct are up to municipal level only, we will create a GADM mapping that is until municipal level. 
As previously mentioned, cleaning would be divided into three parts:
1. exact matching
2. manual cleaning
3. fuzzy matching

After cleaning, uur data frame should have the following headers:
* `region`
* `province`
* `municipality`
* `gadm_province`
* `gadm_municipality`

## first pass cleaning : exact matching

In [51]:
spark.sql('select distinct region, province, municipality from location').count()

1663

In [60]:
spark.sql('''
            SELECT DISTINCT
                l.region, l.province, l.municipality,
                m.NAME_1 gadm_province, m.NAME_2 gadm_municipality
            FROM location l
            LEFT JOIN municipal m
            ON UPPER(m.NAME_1) = l.province
            AND UPPER(m.NAME_2) = l.municipality 
''').createOrReplaceTempView('lookup')
spark.sql('SELECT COUNT(*) FROM lookup').show()

+--------+
|count(1)|
+--------+
|    1663|
+--------+



In [61]:
spark.sql('SELECT count(*) FROM lookup WHERE gadm_province IS NULL').show()

+--------+
|count(1)|
+--------+
|     225|
+--------+



## second pass cleaning: manual replacement

In [62]:
spark.sql('''
            SELECT k.region, k.province, k.municipality, 
                CASE 
                    WHEN region = "NCR"
                        THEN "Metropolitan Manila"
                    WHEN province LIKE "%DAVAO DEL NORTE%"
                        THEN "Davao del Norte"
                    WHEN province LIKE "DAV%OCC%"
                        THEN "Davao del Sur"
                    WHEN province LIKE "COTABATO%NORTH%"
                        THEN "North Cotabato"
                    WHEN province LIKE "%WEST%SAMAR%"
                        THEN "Samar"
                    WHEN region = "OAV"
                        THEN province
                        ELSE p.NAME_1
                END as gadm_province,
            --p.NAME_1 gadm_province, 
                CASE 
                    WHEN region = "OAV"
                        THEN municipality
                    WHEN province LIKE "%MANILA" AND region = "NCR"
                        THEN "Manila"
                        ELSE gadm_municipality
                END as gadm_municipality
            FROM lookup k
            LEFT JOIN provincial p 
            ON (UPPER(p.NAME_1) = k.province OR UPPER(p.VARNAME_1) = k.province)
            --WHERE gadm_province IS NULL
''').createOrReplaceTempView('lookup')
spark.sql('SELECT count(*) null_values FROM lookup WHERE gadm_province IS NULL').show()

+-----------+
|null_values|
+-----------+
|          0|
+-----------+



In [63]:
spark.sql('select count(*) from lookup').show()

+--------+
|count(1)|
+--------+
|    1663|
+--------+



In [64]:
spark.sql('''
            SELECT 
                k.region, 
                k.province, 
                k.municipality, 
                CASE 
                    WHEN k.gadm_province IS NOT NULL
                        THEN gadm_province
                        ELSE m.NAME_1 
                END as gadm_province,
--                m.NAME_1 gadm_province,
                CASE 
                    WHEN province LIKE "%MANILA" AND region = "NCR"
                        THEN "Manila"
                    WHEN region = "OAV"
                        THEN municipality
                        ELSE m.NAME_2 
                END as gadm_municipality
            FROM lookup as k
            LEFT JOIN municipal m
            ON m.NAME_1 = k.gadm_province
            AND (UPPER(m.NAME_2) = k.municipality 
                    OR UPPER(m.VARNAME_2) = k.municipality)
--            WHERE gadm_municipality IS NULL AND province LIKE "%DAV%OCC%"
            ORDER BY region, province, municipality
''').createOrReplaceTempView('lookup')
spark.sql('SELECT count(*) null_values FROM lookup WHERE gadm_municipality IS NULL').show()

+-----------+
|null_values|
+-----------+
|        107|
+-----------+



## third pass cleaning: fuzzy matching

For fuzzy matching, we only consider GADM values which have not been matched as choices to avoid duplication. Since we can't apply a function inside SQL directly, we export the table to rdd first then use the map function. This passes the entire row to the function and performs fuzzy matching inside the function. Because we can't parallelize a sql statement into the rdd map function, we first export our shape tags which have not been mapped into a dataframe. We only have a little over 100 values so this won't take too much memory.

Workflow summary:
1. Determine which gadm values have not been mapped and export to a dataframe. This will be our reference for fuzzy matching later on.
2. Export the lookup table to RDD so we can apply a map function.
3. Create a mapping function that will perform fuzzy matching.

In [74]:
remaining = spark.sql('''
            SELECT NAME_1, NAME_2 
            FROM 
                (
                SELECT m.NAME_1, m.NAME_2, k.gadm_municipality
                FROM municipal m
                LEFT JOIN lookup k
                    ON k.gadm_province = m.NAME_1
                        AND k.gadm_municipality = m.NAME_2)
            WHERE gadm_municipality IS NULL
''').toPandas().groupby('NAME_1').NAME_2.apply(list)

In [76]:
remaining.head()

NAME_1
Abra                                                    [Licuan-Baay]
Agusan del Norte                                        [Mainit Lake]
Agusan del Sur                                         [Bayugan City]
Albay               [Santo Domingo, Bato Lake, Tabaco City, Daraga...
Antique                                              [Tobias Fornier]
Name: NAME_2, dtype: object

In [78]:
def map_municipality(r):
    if r.gadm_municipality:
        return r
    else:
        p = r.gadm_province
        m = r.municipality
        to_return = r
        try:
            replace_with, score = process.extractOne(m, remaining[p])
            if score > 90:
                to_return = Row(r.region, r.province, r.municipality, r.gadm_province, replace_with)
            else:
                if score > 80:
                    if 'DATU' in m or 'SHARIFF' in m:
                        to_return = Row(r.region, r.province, r.municipality, r.gadm_province, r.municipality)
                    else:
                        to_return = Row(r.region, r.province, r.municipality, r.gadm_province, replace_with)
                else:
                    to_return = Row(r.region, r.province, r.municipality, r.gadm_province, r.municipality)
            return to_return
        except:
            return Row(r.region, r.province, r.municipality, r.gadm_province, r.municipality)

In [79]:
rdd = spark.sql('SELECT * FROM lookup').rdd
rdd.map(map_municipality).toDF().createOrReplaceTempView('lookup')

In [80]:
spark.sql('''SELECT * FROM lookup WHERE gadm_municipality IS NULL''').show()

+------+--------+------------+-------------+-----------------+
|region|province|municipality|gadm_province|gadm_municipality|
+------+--------+------------+-------------+-----------------+
+------+--------+------------+-------------+-----------------+



In [83]:
spark.sql('SELECT count(*) FROM lookup').show()

+--------+
|count(1)|
+--------+
|    1663|
+--------+



## export location to gadm lookup

In [85]:
save_query_to_csv('SELECT * FROM lookup', 'location_lookup')

1it [00:00, 236.19it/s]

file saved!


# Create denormalized tables
* replace precinct code with region, province, municipality and barangay in results table.
* we also want to include other useful information in the psgc table.

In [88]:
datasets['president'].limit(2).toPandas()

,precinct_code,contest_code,candidate_name,party_code,votes,col5,ballots_cast,col7,col8,timestamp,pct_votes
0,69140005,199009,"DUTERTE, RODY (PDPLBN)",114,124,3,639,13,8,05/09/2016 17:05:50,0.194053208138
1,69140005,199009,"DEFENSOR SANTIAGO, MIRIAM (PRP)",135,12,2,639,13,8,05/09/2016 17:05:50,0.018779342723


In [104]:
def create_denorm_table(table):
    query = f'''
                SELECT 
                    p.precinct_code, 
                    k.region, 
                    k.gadm_province, 
                    k.gadm_municipality, 
                    g.barangay,
                    p.candidate_name, 
                    p.party_code, 
                    p.votes, 
                    p.ballots_cast, 
                    p.col7 abstain, 
                    p.col8 invalid_votes,
                    p.timestamp,
                    p.pct_votes,
                    g.registered_voters,
                    g.is_city,
                    g.income_class,
                    g.population,
                    g.land_area   
                FROM {table} p
                LEFT JOIN (psgc g 
                    LEFT JOIN lookup k 
                        ON k.province = g.province 
                        AND k.municipality = g.municipality)
                ON g.precinct_code = p.precinct_code
    '''
    table_name = f'denorm_{table[:4]}'
    spark.sql(query).createOrReplaceTempView(table_name)
    save_query_to_csv(f'SELECT * FROM {table_name}', table_name)

In [105]:
for table in ['president', 'vice_president']:
    create_denorm_table(table)

1it [00:00, 212.15it/s]


file saved!


1it [00:00, 253.85it/s]

file saved!


In [106]:
spark.sql('SELECT * FROM denorm_pres LIMIT 2').toPandas()

,precinct_code,region,gadm_province,gadm_municipality,barangay,candidate_name,party_code,votes,ballots_cast,abstain,invalid_votes,timestamp,pct_votes,registered_voters,is_city,income_class,population,land_area
0,10050007,REGION IV-A,Batangas,Batangas City,BARANGAY 5 (POB.),"DUTERTE, RODY (PDPLBN)",114,168,491,2,0,05/09/2016 18:28:46,0.34215885947,602,1.0,1st Class-Special,None,28296.0
1,10050007,REGION IV-A,Batangas,Batangas City,BARANGAY 5 (POB.),"DEFENSOR SANTIAGO, MIRIAM (PRP)",135,35,491,2,0,05/09/2016 18:28:46,0.071283095723,602,1.0,1st Class-Special,None,28296.0


In [107]:
spark.sql('SELECT * FROM denorm_vice WHERE population > 10000 LIMIT 2').toPandas()

,precinct_code,region,gadm_province,gadm_municipality,barangay,candidate_name,party_code,votes,ballots_cast,abstain,invalid_votes,timestamp,pct_votes,registered_voters,is_city,income_class,population,land_area
0,10080008,REGION IV-A,Batangas,Calatagan,BARANGAY 4 (POB.),"MARCOS, BONGBONG (IND)",58,103,436,16,1,05/09/2016 19:08:46,0.23623853211,528,0.0,2nd,51997.0,11200.0
1,10080008,REGION IV-A,Batangas,Calatagan,BARANGAY 4 (POB.),"TRILLANES, ANTONIO IV (IND)",58,6,436,16,1,05/09/2016 19:08:46,0.0137614678899,528,0.0,2nd,51997.0,11200.0
